In [1]:
import numpy as np
import torch
from tqdm import tqdm
import os

In [5]:
'''
choose between train and val
'''
filename = 'val'
# filename = 'train'

In [6]:
block_size = 256
p = 7
N = 1000000
dtype = np.int8
eq_id = p+1
random_list = np.random.randint(low = 1,high =p+1, size = (N,block_size//2), dtype = np.int32)
input_length_list = np.random.randint(low = 1,high =block_size//2, size = (N,), dtype = dtype)
token_id_list = []
for i in range(N):
    current_input_tokens = random_list[i][:input_length_list[i]]
    token_id_list.append(list(current_input_tokens) + [eq_id] + [(current_input_tokens.sum()-1) % p + 1])

In [7]:
arr = np.memmap(filename + '.bin', dtype=dtype, mode='w+', shape=(block_size * len(token_id_list)))
arr_label = np.memmap(filename + '_label.bin', dtype=dtype, mode='w+', shape=(block_size * len(token_id_list)))
print(f"writing {filename}...")
for jj, example in tqdm(enumerate(token_id_list)):
    arr[jj * block_size : jj * block_size + len(example)-1] = example[:-1]
    arr[jj * block_size + len(example)-1 : (jj + 1) * block_size] = 0
    arr_label[jj * block_size : (1+jj) * block_size ] = -1
    arr_label[jj * block_size + input_length_list[jj]] = example[-1]
    # pad to bock_size
arr.flush()
arr_label.flush()

writing val...


1000000it [00:11, 84671.49it/s]
